In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2"

In [2]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [3]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
import datetime

In [4]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train),(x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [5]:
HP_NUM_UNITS = hp.HParam("num_units", hp.Discrete([16, 32]))
HP_DROPOUT = hp.HParam("dropout", hp.RealInterval(0.1, 0.2))
HP_OPTIMIZER = hp.HParam("optimizer", hp.Discrete(["adam", "sgd"]))

METRIC_ACCURACY = "accuracy"

with tf.summary.create_file_writer(f"logs/hparam_tuning").as_default():
    hp.hparams_config(
        hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name="Accuracy")],
    )


In [6]:
def train_test_model(hparams, gpu_bus_id):
    model = tf.keras.models.Sequential(
        [
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation=tf.nn.relu),
            tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
            tf.keras.layers.Dense(10, activation=tf.nn.softmax),
        ]
    )
    model.compile(
        optimizer=hparams[HP_OPTIMIZER],
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )
    with tf.device(f"/device:GPU:{gpu_bus_id}"):
        model.fit(
            x_train,
            y_train,
            epochs=1,
            callbacks=[
                tf.keras.callbacks.TensorBoard(
                    log_dir=f'logs/{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}',
                    histogram_freq=1,
                )
            ],
        )  # Run with 1 epoch to speed things up for demo purposes
        _, accuracy = model.evaluate(x_test, y_test)
    return accuracy


In [7]:
def run(run_dir, hparams, gpu_bus_id):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = train_test_model(hparams, gpu_bus_id)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)


In [18]:
session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
    for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
        for optimizer in HP_OPTIMIZER.domain.values:
            hparams = {
                HP_NUM_UNITS: num_units,
                HP_DROPOUT: dropout_rate,
                HP_OPTIMIZER: optimizer,
            }
            run_name = "run-%d" % session_num
            print("--- Starting trial: %s" % run_name)
            print({h.name: hparams[h] for h in hparams})
            run(f"logs/{run_name}", hparams, gpu_bus_id=2)
            session_num += 1


SyntaxError: EOL while scanning string literal (<ipython-input-18-c92091acdf81>, line 14)

In [9]:
# %tensorboard --logdir f"logs/{NB_NAME}/fit" --port=6008


In [20]:
%reload_ext tensorboard
%tensorboard --logdir logs/ --port=6009
from tensorboard import notebook
notebook.display(port=6009, height=1000)


Reusing TensorBoard on port 6009 (pid 3640), started 0:29:58 ago. (Use '!kill 3640' to kill it.)

Selecting TensorBoard with logdir logs/ (started 0:00:12 ago; port 6009, pid 12064).
